In [1]:
import os
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend([os.path.abspath('..')])

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] on linux


In [2]:
from denoising.helpers import *
from denoising.metrics import Metrics
from nilearn.connectome import sym_matrix_to_vec


In [3]:
atlas = 'HCPex' # Schaefer200 AAL Brainnetome
strategy = 6

In [14]:
trt1 = fetch_ts('/data/Projects/TestRetest_NYU/TRT_outputs',
               #sub=,
               run=1, atlas_name=atlas, strategy=strategy)
trt2 = fetch_ts('/data/Projects/TestRetest_NYU/TRT_outputs',
               #sub=,
               run=2, atlas_name=atlas, strategy=strategy)
trt3 = fetch_ts('/data/Projects/TestRetest_NYU/TRT_outputs',
               #sub=,
               run=3, atlas_name=atlas, strategy=strategy)

no files available: []
no files available: []
no files available: []


In [9]:
fc_trt1 = functional_connectivity(trt1, 'correlation')
fc_trt2 = functional_connectivity(trt2, 'correlation')
fc_trt3 = functional_connectivity(trt3, 'correlation')

In [10]:
vec_trt1 = sym_matrix_to_vec(fc_trt1, discard_diagonal=True)
vec_trt2 = sym_matrix_to_vec(fc_trt2, discard_diagonal=True)
vec_trt3 = sym_matrix_to_vec(fc_trt3, discard_diagonal=True)

In [11]:
metric = Metrics([vec_trt1, vec_trt2, vec_trt3])
print(metric.icc())

0.9886615379245183


In [13]:
metric.bss([vec_trt1, vec_trt2])

array([0.41081952, 0.39872923, 0.3869806 , 0.24791155, 0.27977298,
       0.44881921, 0.34538253, 0.39417706, 0.43733259, 0.13260411,
       0.25950153, 0.39625411, 0.45702798, 0.21196928, 0.40342166,
       0.37266327, 0.36854756, 0.40386829, 0.45047183, 0.38672031,
       0.35793191, 0.41463928])